In [10]:
# import libraries
import os
import pandas as pd
import datetime as dt

In [2]:
# must match a reference ontology
# path in final version will be /opt/data/dim/
concept = pd.read_csv('volumes/data/dim/CONCEPT.csv.gz', header=0, sep='\t', low_memory=False)

In [3]:
concept = concept[['concept_code', 'concept_id']]

In [4]:
concept.drop_duplicates(inplace=True)

In [5]:
concept.rename(columns= {'concept_code': 'cpt'}, inplace=True)

In [6]:
# make a control file to iterate through
# path in final version will be /opt/data/raw/
path = 'volumes/data/raw/'
files = os.listdir(path)
csv_files = [x for x in files if x.endswith(".csv.gz")]

# removes '.csv.gz' (7 characters)
hospital_ids = [int(x[:-7]) for x in csv_files] 
control = pd.DataFrame({'file': csv_files, 'hospital_id': hospital_ids})

In [7]:
#cross reference the control file with the static dimension table
# path in final version will be /opt/dta/dim/
dim = pd.read_csv('volumes/data/dim/hospital.csv', usecols=['hospital_id', 'affiliation'])
control = control.merge(dim, how= 'left', on='hospital_id')
control.sort_values(by='hospital_id', inplace=True, ignore_index=True)

In [8]:
## loop (PENDING)

In [13]:
# print file you are working on now
print(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ' - parsing hospital - ' + str(control.hospital_id[0]))

2021-09-20 10:57:46 - parsing hospital - 1


In [9]:
# read in the data
df = pd.read_csv('volumes/data/raw/' + control.file[0], low_memory=False)

In [14]:
# hospital specific configuration
out = pd.DataFrame({
    'cpt': df['CPT/HCPCS Code'],
    'gross': df['Unit Price'],
    'cash': df['Cash Discount Price'],
    'max': df['Maximum Amount'],
    'min': df['Minimum Amount']
})
out.drop_duplicates(inplace=True)

In [11]:
# if out is not Null loop

In [16]:
# uniform
out.dropna(subset=['cpt'], inplace=True)
out = out[out.cpt != '']
out = out[out.cpt != '*']

In [24]:
# must match a reference code, usually CPT or HCPCS
out = out.merge(concept, on='cpt', sort=True)

In [ ]:
out = out.drop('cpt', axis=1)

In [14]:
# melt from wide to long, and clean
long = pd.melt(out, id_vars='concept_id')
long['value'] = long['value'].str.strip()
long['value'] = long['value'].str.replace(',', '', regex=False)
long['value'] = long['value'].str.replace('[$]', '', regex=False)
long['value'] = long.value.astype(float)
long.dropna(subset=['value'], inplace=True)
long = long[long.value > 0]

# Add hospital id and order columns
# later change the 0 for i inside the loop
long['hospital_id'] = control.hospital_id[0]
long = long[long.columns[[3,0,1,2]]]

In [15]:
# write the data to a flatfile for postgres
# path will be later /opt/data/transformed/
# replace 0 with i inside the loop
if long.shape[0] > 0:
    out_path = 'volumes/data/transformed/' + str(control.hospital_id[0]) + '.csv'
    long.to_csv(out_path, header=False, index=None)

In [16]:
# clear variables from last iteration (inside the loop)
# del out
# del long

In [20]:
col_names = ['hospital_id', 'concept_id', 'variable', 'value']
dfr = pd.read_csv('volumes/data/transformed/1.csv', header=None, names=col_names)

In [21]:
dfr.head()

,hospital_id,concept_id,variable,value
0,1,2718651,gross,447.45
1,1,2718651,gross,50.00
2,1,2718651,gross,193.50
3,1,2718651,gross,900.00
4,1,2718651,gross,52.10


In [22]:
dfn = pd.read_csv('../../NC/hospital-price-transparency/volumes/data/transformed/1.csv', header=None, names=col_names)

In [23]:
dfn.head()

,hospital_id,concept_id,variable,value
0,1,2101826,gross,6147.00
1,1,2101827,gross,9221.00
2,1,40757009,gross,13614.00
3,1,40756966,gross,16559.00
4,1,32598,gross,10.69


In [24]:
dfn.loc[lambda df: df['concept_id'] == 32598]

,hospital_id,concept_id,variable,value
4,1,32598,gross,10.69
5221,1,32598,cash,6.41
10438,1,32598,max,6.30
15624,1,32598,min,6.30


In [25]:
dfr.loc[lambda df: df['concept_id'] == 32598]

,hospital_id,concept_id,variable,value
3715,1,32598,gross,10.69
8932,1,32598,cash,6.41
14118,1,32598,max,6.30
19304,1,32598,min,6.30
